In [1]:
import sys
import os 
import openmc
import numpy as np
import matplotlib.pyplot as plt
import openmc.mgxs as mgxs
import pandas as pd
from IPython.display import Image
import glob

### Import Output datas

In [2]:
sp_file='/home/pablo/internship/perso/openmc/data_generation/statepoint_ce.h5'
sm_file='/home/pablo/internship/perso/openmc/data_generation/summary_ce.h5'
sm=openmc.Summary(sm_file)
sp = openmc.StatePoint(sp_file)
sp.link_with_summary(sm)
model=openmc.model.Model
model_file='/home/pablo/internship/perso/openmc/data_generation/model.xml'
model=openmc.model.Model.from_model_xml(model_file)

/home/pablo/anaconda3/envs/openmc-env/lib/python3.12/site-packages/openmc/mixin.py:70: IDWarning: Another Material instance already exists with id=1.
  warn(msg, IDWarning)
/home/pablo/anaconda3/envs/openmc-env/lib/python3.12/site-packages/openmc/mixin.py:70: IDWarning: Another Material instance already exists with id=2.
  warn(msg, IDWarning)
/home/pablo/anaconda3/envs/openmc-env/lib/python3.12/site-packages/openmc/mixin.py:70: IDWarning: Another Material instance already exists with id=3.
  warn(msg, IDWarning)
/home/pablo/anaconda3/envs/openmc-env/lib/python3.12/site-packages/openmc/mixin.py:70: IDWarning: Another Surface instance already exists with id=1.
  warn(msg, IDWarning)
/home/pablo/anaconda3/envs/openmc-env/lib/python3.12/site-packages/openmc/mixin.py:70: IDWarning: Another Surface instance already exists with id=2.
  warn(msg, IDWarning)
/home/pablo/anaconda3/envs/openmc-env/lib/python3.12/site-packages/openmc/mixin.py:70: IDWarning: Another Surface instance already exists

### Recreate Library

In [3]:
mgxs_lib = openmc.mgxs.Library(model.geometry)
groups = openmc.mgxs.EnergyGroups(openmc.mgxs.GROUP_STRUCTURES['XMAS-172'])
mgxs_lib.energy_groups = groups
mgxs_lib.scatter_format = "legendre"
mgxs_lib.legendre_order = 3
mgxs_lib.mgxs_types = ['total', 'absorption', 'fission', 'nu-fission', 'chi', 'scatter matrix', 'nu-scatter matrix', 'inverse-velocity']
mgxs_lib.tally_trigger =openmc.Trigger('std_dev', 1e-2)
mgxs_lib.by_nuclide = False
mgxs_lib.domain_type = 'cell'
mgxs_lib.domains = model.geometry.get_all_material_cells().values()
mgxs_lib.build_library()
mgxs_lib.load_from_statepoint(sp)


/home/pablo/anaconda3/envs/openmc-env/lib/python3.12/site-packages/openmc/mgxs/library.py:401: RuntimeWarning: The P0 correction will be ignored since the scattering order 3 is greater than zero
  warn(msg, RuntimeWarning)


mgxs_file='/home/pablo/internship/perso/openmc/data_generation/mgxs.h5'
openmc.MGXSLibrary.from_hdf5(mgxs_file)
print(openmc.MGXSLibrary.xsdatas)

### Post Processing

In [4]:
def cell_domain(cell_name):
    for k in range(len(mgxs_lib.domains)):
        if mgxs_lib.domains[k].name==cell_name:
            return(mgxs_lib.domains[k].id)

In [5]:
def mgxs_value(X,reaction,cell_type):  
    cell_name=cell_type+'_'+str(X[0])+'_'+str(X[1])
    cell=cell_domain(cell_name)
    mgxs_plot = mgxs_lib.get_mgxs(cell, reaction)
    mgxs=mgxs_plot.get_pandas_dataframe(xs_type='macro')['mean'].values
    return(mgxs)

In [6]:
mgxs_value((0,0),'fission','fuel')

array([3.05802987e-02, 2.91275214e-02, 2.65976512e-02, 2.29294364e-02,
       2.28108959e-02, 2.29910014e-02, 2.17384163e-02, 1.70662918e-02,
       1.30804906e-02, 1.26764773e-02, 1.27563460e-02, 1.23848879e-02,
       1.24577109e-02, 1.26719530e-02, 1.25864115e-02, 1.14335035e-02,
       7.33396966e-03, 2.42014994e-03, 1.51484289e-03, 1.31248752e-03,
       1.19655605e-03, 9.63297205e-04, 8.02957527e-04, 7.67064182e-04,
       7.67262030e-04, 7.75860440e-04, 8.06227102e-04, 8.23034996e-04,
       8.30801071e-04, 8.52143386e-04, 8.88494004e-04, 9.57937330e-04,
       1.01085755e-03, 1.05981829e-03, 1.12337227e-03, 1.18642180e-03,
       1.23571726e-03, 1.29660070e-03, 1.35650290e-03, 1.39772831e-03,
       1.42334284e-03, 1.55078855e-03, 1.66217250e-03, 1.82755230e-03,
       2.02214398e-03, 2.08944988e-03, 2.28573042e-03, 2.61516449e-03,
       2.94629709e-03, 3.21299889e-03, 3.52808078e-03, 3.65274960e-03,
       4.29768154e-03, 3.95100073e-03, 5.28191031e-03, 5.81846097e-03,
      